In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key


In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Mataura,NZ,2021-11-09 11:58:57,-46.1927,168.8643,56.23,76,100,2.01
1,1,San Patricio,US,2021-11-09 11:58:58,28.0170,-97.5169,73.24,80,87,5.12
2,2,Awbari,LY,2021-11-09 11:58:58,26.5921,12.7805,83.62,13,100,7.56
3,3,Ushuaia,AR,2021-11-09 11:58:58,-54.8000,-68.3000,46.06,61,0,5.75
4,4,Bluff,NZ,2021-11-09 11:58:59,-46.6000,168.3333,55.72,88,100,4.23


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:

# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
fig



Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [8]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
2,2,Awbari,LY,2021-11-09 11:58:58,26.5921,12.7805,83.62,13,100,7.56
14,14,Kanniyakumari,IN,2021-11-09 11:59:03,8.0833,77.5667,78.94,80,100,9.31
20,20,Hit,IQ,2021-11-09 11:59:05,33.6416,42.8251,78.94,18,6,6.76
24,24,Cape Town,ZA,2021-11-09 11:58:26,-33.9258,18.4232,75.11,46,20,3.00
25,25,Ciudad Bolivar,VE,2021-11-09 11:59:07,8.1222,-63.5497,77.86,78,43,9.17
26,26,Atuona,PF,2021-11-09 11:58:33,-9.8000,-139.0333,77.61,75,10,14.76
27,27,Vaini,TO,2021-11-09 11:59:08,-21.2000,-175.2000,75.36,100,90,3.44
30,30,Chauk,MM,2021-11-09 11:59:09,20.8833,94.8167,79.63,64,37,5.30
34,34,Cayenne,GF,2021-11-09 11:59:11,4.9333,-52.3333,78.84,89,75,8.01
36,36,Luderitz,NaN,2021-11-09 11:59:12,-26.6481,15.1594,75.36,45,0,24.85


In [9]:
preferred_cities_df.count()

City_ID       165
City          165
Country       162
Date          165
Lat           165
Lng           165
Max Temp      165
Humidity      165
Cloudiness    165
Wind Speed    165
dtype: int64

In [10]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Awbari,LY,83.62,26.5921,12.7805,
14,Kanniyakumari,IN,78.94,8.0833,77.5667,
20,Hit,IQ,78.94,33.6416,42.8251,
24,Cape Town,ZA,75.11,-33.9258,18.4232,
25,Ciudad Bolivar,VE,77.86,8.1222,-63.5497,
26,Atuona,PF,77.61,-9.8000,-139.0333,
27,Vaini,TO,75.36,-21.2000,-175.2000,
30,Chauk,MM,79.63,20.8833,94.8167,
34,Cayenne,GF,78.84,4.9333,-52.3333,
36,Luderitz,NaN,75.36,-26.6481,15.1594,


In [11]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [12]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotel = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Awbari,LY,83.62,26.5921,12.7805,فندق أوباري
14,Kanniyakumari,IN,78.94,8.0833,77.5667,Sparsa
20,Hit,IQ,78.94,33.6416,42.8251,صالون زيرو
24,Cape Town,ZA,75.11,-33.9258,18.4232,Southern Sun Waterfront Cape Town
25,Ciudad Bolivar,VE,77.86,8.1222,-63.5497,Hotel Edi
...,...,...,...,...,...,...
562,Isangel,VU,76.87,-19.5500,169.2667,Tanna Lodge
568,Souillac,MU,80.37,-20.5167,57.5167,Shanti Maurice Resort & Spa
569,Vila Do Maio,CV,81.39,15.1333,-23.2167,Hotel Marilu
570,Luau,AO,89.73,-10.7073,22.2247,Casa- Bernardo Cândido


In [13]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [16]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))